In [23]:
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa

from keras import backend as K
from keras.models import Model

from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Lambda, concatenate

In [2]:
import glob
import h5py
import itertools

''' Load dataset '''
trainPath = './dataset/SeqTrain/'
valPath = './dataset/SeqVal/'


trainFile = glob.glob(trainPath + '*.h5')
valFile = glob.glob(valPath + '*.h5')

data = h5py.File(trainFile[0], 'r')

In [3]:
# def batchGenerator(file_names, batch_size = 3):
   
#     batch_x = np.zeros((batch_size, 88, 200, 3))    
#     batch_y = np.zeros((batch_size, 28))
    
#     while True:
#         for i in range(batch_size - 1):
#             file_idx = np.random.randint(len(file_names) - 1)
#             sample_idx = np.random.randint(200-1)

#             data = h5py.File(file_names[file_idx], 'r')

#             batch_x[i] = data['rgb'][sample_idx]
#             batch_y[i] = data['targets'][sample_idx]
#             data.close()

#         yield (batch_x, batch_y)

def branchGenerator(file_names, branchNum, batch_size = 3):
    batch_x = np.zeros((batch_size, 88, 200, 3))    
    batch_y = np.zeros((batch_size, 28))
          
    while True: 
        
        for i in range(batch_size - 1):     
            
            file_idx = np.random.randint(len(file_names) - 1) 
            sample_idx = np.random.randint(200-1)
            
            data = h5py.File(file_names[file_idx], 'r')
       
            ''' High level command (Branching): 2 - Follow lane, 3 - Left, 4 - Right, 5 - Straight '''
        
#             if data['targets'][sample_idx][24] == branchNum:
                
            batch_x[i] = data['rgb'][sample_idx]
            batch_y[i] = data['targets'][sample_idx][24] #% 4  # To shift values 4 and 5 to apply masking

            data.close()
                
        yield (batch_x, batch_y)

### Data Augmentation

In [4]:
st = lambda aug: iaa.Sometimes(0.4, aug)
oc = lambda aug: iaa.Sometimes(0.3, aug)
rl = lambda aug: iaa.Sometimes(0.09, aug)

seq = iaa.Sequential([
        rl(iaa.GaussianBlur((0, 1.5))),                                               # blur images with a sigma between 0 and 1.5
        rl(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05), per_channel=0.5)),     # add gaussian noise to images
        oc(iaa.Dropout((0.0, 0.10), per_channel=0.5)),                                # randomly remove up to X% of the pixels
        oc(iaa.CoarseDropout((0.0, 0.10), size_percent=(0.08, 0.2),per_channel=0.5)), # randomly remove up to X% of the pixels
        oc(iaa.Add((-40, 40), per_channel=0.5)),                                      # adjust brightness of images (-X to Y% of original value)
        st(iaa.Multiply((0.10, 2.5), per_channel=0.2)),                               # adjust brightness of images (X -Y % of original value)
        rl(iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5)),                   # adjust the contrast
], random_order=True)

## Conditional Imitation Learning Network

In [5]:
import tensorflow as tf

branch_config = [
                    ["Steer", "Gas", "Brake"], ["Steer", "Gas", "Brake"], 
                    ["Steer", "Gas", "Brake"], ["Steer", "Gas", "Brake"],
                    ["Speed"] 
              ]

image_size = (88, 200, 3)

input_image = [image_size[0], image_size[1], image_size[2]]

input_data = []

input_data.append(tf.placeholder(tf.float32, shape=[None, 4], name="input_control"))

input_data.append(tf.placeholder(tf.float32, shape=[None, 1], name="input_speed"))

target_speed = tf.placeholder(tf.float32, shape=[None, 1], name="target_speed")

target_controller = tf.placeholder(tf.float32, shape=[None, 3], name="target_control")




In [58]:
branches = []

def conv_block(inputs, filters, kernel_size, strides):
    x = Conv2D(filters, (kernel_size, kernel_size), strides = strides, activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(1,1), strides=(1,1))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    return x

def fc_block(inputs, units):
    fc = Dense(units, activation = 'relu')(inputs)
    fc = Dropout(0.2)(fc)
    return fc


inputs = Input(input_image)

'''inputs, filters, kernel_size, strides'''

""" Conv 1 """
x = conv_block(inputs, 32, 5, 2)
x = conv_block(x, 32, 3, 1)

""" Conv 2 """
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 1)

""" Conv 3 """
x = conv_block(x, 128, 3, 2)
x = conv_block(x, 128, 3, 1)

""" Conv 4 """
x = conv_block(x, 256, 3, 1)
x = conv_block(x, 256, 3, 1)

""" Reshape """
x = Flatten()(x)

""" FC1 """
x = fc_block(x, 512)

""" FC2 """
x = fc_block(x, 512)

"""Process Control"""

""" Speed (measurements) """
speed = input_data[1] # input_speed
speed = fc_block(speed, 128)
speed = fc_block(speed, 128)

""" Joint sensory """
j = concatenate([x, speed])
j = fc_block(j, 512)

for i in range(len(branch_config)):
    
    # Use only image input to predict the speed
    
    if branch_config[i][0] == "Speed":
        branch_output = fc_block(x, 256)
        branch_output = fc_block(branch_output, 256)

    else:
        branch_output = fc_block(j, 256)
        branch_output = fc_block(branch_output, 256)

    fully_connected = Dense(len(branch_config[i]), activation = 'relu')(branch_output)
   
    branches.append(fully_connected)
    
# print(fully_connected)

model = Model(inputs=inputs,outputs=branches[0])

AttributeError: 'NoneType' object has no attribute '_inbound_nodes'

In [50]:
branch_output

<tf.Tensor 'dropout_759/cond/Merge:0' shape=(?, 256) dtype=float32>

In [36]:

# model.summary()

# model2 = Model(inputs=speed)
# model2.summary()



In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

### Control Network?

In [ ]:
for i in range(len(branch_config)): 
    print(branch_config[i][0])
    if branch_config[i][0] == "Speed":
        speed_model = Model(inputs=networkBranch[-1], outputs=_target_speed)
    else:
        branch_model = Model(inputs=networkBranch[i], outputs=_target_controller)


In [ ]:
# load_CIL_network()
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='auto')

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00002))



model.fit_generator(batchGenerator(trainFile),          
                    max_queue_size=1,
                    epochs=5,
                    steps_per_epoch=50,
                    validation_data=batchGenerator(valFile),
                    validation_steps=len(valFile),
                    callbacks=[checkpoint],
                    verbose=1)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
import tensorflow as tf
input_shape = [None, 28, 28, 10]
x = tf.placeholder( tf.float32, input_shape, name='x')

print(tf.shape(x))
print(x.get_shape()[-1])